In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
import random
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

Load Data

In [6]:
df = pd.read_csv (
  '/content/drive/MyDrive/code/sentiment-analysis-with-deep-learning/Data/smile-annotations-final.csv',
  names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

Explore Data

In [7]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [9]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

Clean Data

In [10]:
df = df[~df.category.str.contains('\|')]

In [11]:
df = df[df.category != 'nocode']

In [12]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [13]:
possible_labels = df.category.unique ()

Label Dict

In [14]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label] = index

In [15]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [16]:
df['label'] = df.category.replace(label_dict)
df.head()

<ipython-input-16-be17be7dd7a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df.category.replace(label_dict)


,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


Split Data

In [17]:
X_train, X_val, y_train, y_val = train_test_split(
  df.index.values,
  df.label.values,
  test_size=0.15,
  random_state=17,
  stratify=df.label.values
)

In [18]:
df['data_type'] = ['not_set']*df.shape[0]

<ipython-input-18-6709a8c15703>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_type'] = ['not_set']*df.shape[0]


In [19]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [20]:
df.groupby(['category', 'label', 'data_type'])

Tokenize Data

In [22]:
tokenizer = BertTokenizer.from_pretrained(
  "bert-base-uncased",
  do_lower_case=True
)

In [23]:
encoded_data_train = tokenizer.batch_encode_plus(
  df[df.data_type=='train'].text.values,
  add_special_tokens=True,
  return_attention_mask=True,
  pad_to_max_length=True,
  max_length=256,
  return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
  df[df.data_type=='val'].text.values,
  add_special_tokens=True,
  return_attention_mask=True,
  pad_to_max_length=True,
  max_length=256,
  return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
dataset_train = TensorDataset(input_ids_train,
                            attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val, labels_val)

In [25]:
len(dataset_train)

1258

In [26]:
len(dataset_val)

223

Create Model

In [27]:
model = BertForSequenceClassification.from_pretrained(
  'bert-base-uncased',
  num_labels = len(label_dict),
  output_attentions=False,
  output_hidden_states=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data Loaders

In [28]:
batch_size = 4 #32
dataloader_train = DataLoader(
  dataset_train,
  sampler = RandomSampler(dataset_train),
  batch_size=batch_size,
)
dataloader_val = DataLoader(
  dataset_val,
  sampler = RandomSampler(dataset_val),
  batch_size=32
)

Opizmizer

In [29]:
optimizer = AdamW(
  model.parameters(),
  lr=1e-5, #2e-5 > 5e-5
  eps=1e-8
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Scheduler

In [30]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=len(dataloader_train)*epochs
)

Metrics

In [31]:
import numpy as np

In [32]:
from sklearn.metrics import f1_score

In [54]:
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten ()
  labels_flat = labels.flatten ()
  return f1_score(labels_flat, preds_flat, average='weighted')

In [34]:
def accuracy_per_class(preds, labels):
  label_dict_inverse = {v: k for k, v in label_dict.items ()}
  preds_flat = np.argmax (preds, axis=1). flatten ()
  labels_flat = labels.flatten ()
  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len (y_preds[y_preds==label])}/{len(y_true)}\n')

CUDA

In [35]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [36]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print (device)

cuda


Evaluation Functions

In [37]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals


Train

In [38]:
for epoch in tqdm(range(1, epochs+1)):
  model.train ()
  loss_train_total = 0
  progress_bar = tqdm(dataloader_train,
  desc='Epoch {:1d}'. format (epoch), leave=False, disable=False)
  for batch in progress_bar:
    model.zero_grad()
    batch = tuple(b.to(device) for b in batch)
    inputs = {
      'input_ids':batch[0],
      'attention_mask' : batch[1],
      'labels':batch[2]
    }
    outputs = model (**inputs)
  loss = outputs [0]
  loss_train_total += loss.item ()
  loss.backward ()
  torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
  optimizer.step ()
  scheduler.step ()
  progress_bar.set_postfix({'training loss': '{:.3f}'.format(loss.item()/len(batch))})

torch.save(model.state_dict(), f'/content/drive/MyDrive/code/sentiment-analysis-with-deep-learning/Models/BERT_ft_epoch{epoch}.model')
tqdm.write("\nEpoch {epoch}")
loss_train_avg = loss_train_total/len(dataloader_train)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]

<ipython-input-38-86658ba88a9d>:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)


Epoch 2:   0%|          | 0/315 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/315 [00:00<?, ?it/s]

...

Epoch 7:   0%|          | 0/315 [00:00<?, ?it/s]

Epoch 8:   0%|          | 0/315 [00:00<?, ?it/s]

Epoch 9:   0%|          | 0/315 [00:00<?, ?it/s]

Epoch 10:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}


Evaluation

In [39]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
model.to(device)
pass

In [41]:
model.load_state_dict(
  torch.load('/content/drive/MyDrive/code/sentiment-analysis-with-deep-learning/Models/finetuned_bert_epoch_1_gpu_trained.model',
    map_location=torch.device('cpu')))

<All keys matched successfully>

In [42]:
_, predictions, true_vals = evaluate(dataloader_val)

Accuracy

In [52]:
sum(map(lambda zip: zip[0]==zip[1], zip(map(np.argmax,predictions), true_vals)))/len(predictions)

0.8789237668161435

In [55]:
f1_score_func(predictions, true_vals)

0.8758551820295151

In [43]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 163/171

Class: not-relevant
Accuracy: 20/32

Class: angry
Accuracy: 7/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 4/5

Class: surprise
Accuracy: 2/5

